In [ ]:
import numpy as np
import pandas as pd

def benchmark(x1, x2):
    return (    (1.3356 * (1.5 * (1 - x1))) 
                + (np.exp((2 * x1) - 1) * np.sin((3 * np.pi) * ((x1 - 0.6) ** 2)))
                + (np.exp(3 * (x2 - 0.5)) * np.sin((4 * np.pi) * ((x2 - 0.9) ** 2)))
            )

def getData(grid):
    x1 = np.linspace(0, 1, grid)
    x2 = np.linspace(0, 1, grid)
    x1, x2 = np.meshgrid(x1, x2)
    f_x1_x2 = benchmark(x1, x2)
    data = {'x1': x1.flatten(), 'x2': x2.flatten(), 'f(x1,x2)': f_x1_x2.flatten()}
    df = pd.DataFrame(data)
    return df


df_1000 = getData(32)
df_test = pd.read_excel("../Kriging-data.xlsx", sheet_name="Test")
df_training = pd.read_excel("../Kriging-data.xlsx", sheet_name="Training")

df_virtual = pd.read_excel("../VirtualSamples.xlsx", sheet_name="Gaussian")

In [ ]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

scaler = StandardScaler()
out_scaler = StandardScaler()

def show_norm(df, label="data", plot=False):
    df_norm = pd.DataFrame(scaler.transform(df), columns=df.columns)
    df_denorm = pd.DataFrame(scaler.inverse_transform(df_norm), columns=df_norm.columns)

    if (plot):
        df.plot(title=f"{label}: Original data")
        df_norm.plot(title=f"{label}: Normalized data")
        df_denorm.plot(title=f"{label}: Denormalized data")
    return (df_norm)


def test_out_scaler(df):
    out = df["f(x1,x2)"].values.reshape(-1, 1)  
    plt.plot(out, label='Original')
    out_scaler.fit(out)
    norm = out_scaler.transform(out)
    plt.plot(norm, label='Normalizado')
    plt.plot(out_scaler.inverse_transform(norm), label='desnormalizado')
    plt.legend()
    plt.show()

scaler.fit(df_training)
test_out_scaler(df_training)

df_training_norm = show_norm(df_training, "Training")
df_1000_norm = show_norm(df_1000)
df_virtual_norm = show_norm(pd.concat([df_training, df_virtual]))

In [ ]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [ ]:
def split_df(df):
    _input = np.vstack([df['x1'], df['x2']]).T
    _output = np.array(df['f(x1,x2)'])
    return (_input, _output)

In [23]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def compute_k(self):
        total_parameters = 0
        for layer in self.model.layers:
            weights = layer.get_weights()
            if len(weights) > 0:  
                for w in weights:
                    total_parameters += np.prod(w.shape)
        return total_parameters
        
    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer

        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

In [24]:
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error, mean_absolute_percentage_error 

class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
        self.k = 0

    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)
      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output, sup_input, sup_output):
      input_train, input_val, output_train, output_val = train_test_split(input, output, test_size=0.3, shuffle = True)
      
      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.sup_dataset, self.sup_input, self.sup_output = self.create_dataset(sup_input, sup_output)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._data = (input, output)
      self._train = (input_train, output_train)
      self._val = (input_val, output_val)      
      self._sup = (sup_input, sup_output)

    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)
    
    def get_new_metrics(self, orig, pred, r2, mse):
      n = len(orig) # N: quantidade de saidas
      k = self.k
      waste = (orig.flatten() - pred.flatten())

      mape = mean_absolute_percentage_error(orig, pred)  
      r2_adj = 1 - (((n - 1)/(n - k - 1)) * (1 - r2))
      rsd = np.sqrt(np.sum(waste ** 2) / (n - 2))
      rmse = root_mean_squared_error(orig, pred)          
      aic = (-2 * np.log(mse)) + (2 * k)
      bic = (-2 * np.log(mse)) + (k * np.log(n))
      return (mape, r2_adj, rsd, rmse, aic, bic)
      

    def get_metrics(self):
          # Calculando a saida com os dados normalizados
          pred = self.lm_model.predict(self.input).flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          sup_pred = self.lm_model.predict(self.sup_input).flatten()
          # Calculando as metricas com a saida desnormalizada
          pred_denorm = out_scaler.inverse_transform(pred.reshape(-1, 1))
          val_pred_denorm = out_scaler.inverse_transform(val_pred.reshape(-1, 1))
          sup_pred_denorm = out_scaler.inverse_transform(sup_pred.reshape(-1, 1))
          out_denorm = out_scaler.inverse_transform(self._data[1].reshape(-1, 1))
          val_denorm = out_scaler.inverse_transform(self._val[1].reshape(-1, 1))
          sup_denorm = out_scaler.inverse_transform(self._sup[1].reshape(-1, 1))

          r2 = r2_score(out_denorm, pred_denorm)
          r2_val = r2_score(val_denorm, val_pred_denorm)
          r2_sup = r2_score(sup_denorm,  sup_pred_denorm)
          mse = mean_squared_error(out_denorm, pred_denorm)          
          mse_val = mean_squared_error(val_denorm, val_pred_denorm)
          mse_sup = mean_squared_error(sup_denorm,  sup_pred_denorm)
          mape, r2_adj, rsd, rmse, aic, bic = self.get_new_metrics(out_denorm, pred_denorm, r2, mse)
          metrics = {
                          'r2': r2,
                          'r2_sup': r2_sup,                          
                          'r2_val': r2_val,                          
                          'mse': mse,
                          'mse_sup': mse_sup,
                          'mse_val': mse_val,                          
                          'mape': mape,
                          'rmse': rmse,
                          'r2_adj': r2_adj,
                          'rsd': rsd,
                          'aic': aic,
                          'bic': bic
                          }

          return metrics

In [25]:
import pickle
from itertools import product

class Tester:
  def __init__(self, _df, _df_1000,  run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = split_df(_df)
    self.input_1000, self.output_1000 = split_df(_df_1000)
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=2,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)
    trainer.k = shuffler.compute_k()

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._data, trainer._train,  trainer._val), f)
      
  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path)
  
  def SaveInitialModelWeights(self, model, weights, fileName):
    model.set_weights(weights)
    self.SaveModelWeights(model, f"initial-weights/{fileName}")

  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      initial_weights = trainer.lm_model.get_weights()
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie):
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.SaveInitialModelWeights(model, initial_weights, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    return(save)

  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output, self.input_1000, self.output_1000)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults('mse_sup', header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

# Treinando apenas com dados originais


In [26]:
tester = Tester(_df=df_training_norm,
                _df_1000=df_1000_norm,
                run_times=50, dataset_run_times=10)
tester.Loop(sort_by='r2',
            boundarie = 0.8,
            hidden_sizes = [[12, 2]],
            regularizers=[0.02],
            learning_rate=[0.01])

Testando combinacao1: Hidden Size=[12, 2], regularizer=0.02, learning_rate=0.01
+++++++++++ [1_0] | 1 ++++++++++++++++++
Stopped at epoch:  8
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Stopped at epoch:  91
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Stopped at epoch:  148
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Stopped at epoch:  212
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Stopped at epoch:  176
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Stopped at epoch:  143
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_0] | 7 ++++++++++++++++++
Stopped at epoch:  125
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_0] | 8 ++++++++++++++++++
Stopped at epoch:  80
32/32 [==============================] - 

,r2,r2_sup,r2_val,mse,mse_sup,mse_val,mape,rmse,r2_adj,rsd,aic,bic
model_1_0_6,0.806169,0.591139,0.844280,0.099113,0.242718,0.058915,0.159846,0.314822,1.156142,0.325872,134.622989,225.700819
model_1_0_5,0.910174,0.560966,0.883207,0.045931,0.260629,0.044188,0.096084,0.214316,1.072360,0.221838,136.161221,227.239051
model_1_0_0,0.860649,0.556042,0.893936,0.071255,0.263553,0.040128,0.150031,0.266936,1.112255,0.276305,135.282982,226.360812
model_1_0_24,0.803054,0.553448,0.856919,0.100706,0.265093,0.054134,0.154515,0.317342,1.158651,0.328480,134.591107,225.668937
model_1_0_23,0.803054,0.553448,0.856919,0.100706,0.265093,0.054134,0.154515,0.317342,1.158651,0.328480,134.591107,225.668937
model_1_0_22,0.803054,0.553448,0.856919,0.100706,0.265093,0.054134,0.154515,0.317342,1.158651,0.328480,134.591107,225.668937
model_1_0_21,0.803054,0.553448,0.856919,0.100706,0.265093,0.054134,0.154515,0.317342,1.158651,0.328480,134.591107,225.668937
model_1_0_20,0.803054,0.553448,0.856919,0.100706,0.265093,0.054134,0.154515,0.317342,1.158651,0.328480,134.591107,225.668937
model_1_0_19,0.803054,0.553448,0.856919,0.100706,0.265093,0.054134,0.154515,0.317342,1.158651,0.328480,134.591107,225.668937
model_1_0_18,0.803054,0.553448,0.856919,0.100706,0.265093,0.054134,0.154515,0.317342,1.158651,0.328480,134.591107,225.668937


DataFrame salvo em ../results/metrics_1_0
+++++++++++ [1_1] | 1 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_1] | 2 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 3ms/step
+++++++++++ [1_1] | 3 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_1] | 4 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_1] | 5 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_1] | 6 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_1] | 7 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_1] | 8 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_1] | 9 ++++++++++++++++

,r2,r2_sup,r2_val,mse,mse_sup,mse_val,mape,rmse,r2_adj,rsd,aic,bic
model_1_1_0,0.801703,0.553949,0.445852,0.101396,0.264795,0.172825,0.154881,0.318428,1.159739,0.329604,134.577437,225.655266
model_1_1_27,0.801703,0.553949,0.445852,0.101396,0.264795,0.172825,0.154881,0.318428,1.159739,0.329604,134.577437,225.655266
model_1_1_28,0.801703,0.553949,0.445852,0.101396,0.264795,0.172825,0.154881,0.318428,1.159739,0.329604,134.577437,225.655266
model_1_1_29,0.801703,0.553949,0.445852,0.101396,0.264795,0.172825,0.154881,0.318428,1.159739,0.329604,134.577437,225.655266
model_1_1_30,0.801703,0.553949,0.445852,0.101396,0.264795,0.172825,0.154881,0.318428,1.159739,0.329604,134.577437,225.655266
model_1_1_31,0.801703,0.553949,0.445852,0.101396,0.264795,0.172825,0.154881,0.318428,1.159739,0.329604,134.577437,225.655266
model_1_1_32,0.801703,0.553949,0.445852,0.101396,0.264795,0.172825,0.154881,0.318428,1.159739,0.329604,134.577437,225.655266
model_1_1_33,0.801703,0.553949,0.445852,0.101396,0.264795,0.172825,0.154881,0.318428,1.159739,0.329604,134.577437,225.655266
model_1_1_34,0.801703,0.553949,0.445852,0.101396,0.264795,0.172825,0.154881,0.318428,1.159739,0.329604,134.577437,225.655266
model_1_1_35,0.801703,0.553949,0.445852,0.101396,0.264795,0.172825,0.154881,0.318428,1.159739,0.329604,134.577437,225.655266


DataFrame salvo em ../results/metrics_1_1
+++++++++++ [1_2] | 1 ++++++++++++++++++
Stopped at epoch:  9
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_2] | 2 ++++++++++++++++++
Stopped at epoch:  9
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_2] | 3 ++++++++++++++++++
Stopped at epoch:  9
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_2] | 4 ++++++++++++++++++
Stopped at epoch:  9
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_2] | 5 ++++++++++++++++++
Stopped at epoch:  9
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_2] | 6 ++++++++++++++++++
Stopped at epoch:  9
32/32 [==============================] - 0s 3ms/step
+++++++++++ [1_2] | 7 ++++++++++++++++++
Stopped at epoch:  9
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_2] | 8 ++++++++++++++++++
Stopped at epoch:  9
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_2] | 9 ++++++++++++++++

...

DataFrame salvo em ../results/metrics_1_7
+++++++++++ [1_8] | 1 ++++++++++++++++++
Stopped at epoch:  12
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_8] | 2 ++++++++++++++++++
Stopped at epoch:  12
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_8] | 3 ++++++++++++++++++
Stopped at epoch:  12
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_8] | 4 ++++++++++++++++++
Stopped at epoch:  12
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_8] | 5 ++++++++++++++++++
Stopped at epoch:  12
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_8] | 6 ++++++++++++++++++
Stopped at epoch:  12
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_8] | 7 ++++++++++++++++++
Stopped at epoch:  12
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_8] | 8 ++++++++++++++++++
Stopped at epoch:  12
32/32 [==============================] - 0s 3ms/step
+++++++++++ [1_8] | 9 ++++++++

,r2,r2_sup,r2_val,mse,mse_sup,mse_val,mape,rmse,r2_adj,rsd,aic,bic
model_1_8_0,0.828268,0.593636,0.654696,0.087813,0.241235,0.089166,0.146444,0.296332,1.13834,0.306733,134.865098,225.942928
model_1_8_27,0.828268,0.593636,0.654696,0.087813,0.241235,0.089166,0.146444,0.296332,1.13834,0.306733,134.865098,225.942928
model_1_8_28,0.828268,0.593636,0.654696,0.087813,0.241235,0.089166,0.146444,0.296332,1.13834,0.306733,134.865098,225.942928
model_1_8_29,0.828268,0.593636,0.654696,0.087813,0.241235,0.089166,0.146444,0.296332,1.13834,0.306733,134.865098,225.942928
model_1_8_30,0.828268,0.593636,0.654696,0.087813,0.241235,0.089166,0.146444,0.296332,1.13834,0.306733,134.865098,225.942928
model_1_8_31,0.828268,0.593636,0.654696,0.087813,0.241235,0.089166,0.146444,0.296332,1.13834,0.306733,134.865098,225.942928
model_1_8_32,0.828268,0.593636,0.654696,0.087813,0.241235,0.089166,0.146444,0.296332,1.13834,0.306733,134.865098,225.942928
model_1_8_33,0.828268,0.593636,0.654696,0.087813,0.241235,0.089166,0.146444,0.296332,1.13834,0.306733,134.865098,225.942928
model_1_8_34,0.828268,0.593636,0.654696,0.087813,0.241235,0.089166,0.146444,0.296332,1.13834,0.306733,134.865098,225.942928
model_1_8_35,0.828268,0.593636,0.654696,0.087813,0.241235,0.089166,0.146444,0.296332,1.13834,0.306733,134.865098,225.942928


DataFrame salvo em ../results/metrics_1_8
+++++++++++ [1_9] | 1 ++++++++++++++++++
Stopped at epoch:  9
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_9] | 2 ++++++++++++++++++
Stopped at epoch:  9
32/32 [==============================] - 0s 3ms/step
+++++++++++ [1_9] | 3 ++++++++++++++++++
Stopped at epoch:  9
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_9] | 4 ++++++++++++++++++
Stopped at epoch:  9
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_9] | 5 ++++++++++++++++++
Stopped at epoch:  9
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_9] | 6 ++++++++++++++++++
Stopped at epoch:  9
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_9] | 7 ++++++++++++++++++
Stopped at epoch:  9
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_9] | 8 ++++++++++++++++++
Stopped at epoch:  9
32/32 [==============================] - 0s 2ms/step
+++++++++++ [1_9] | 9 ++++++++++++++++

,r2,r2_sup,r2_val,mse,mse_sup,mse_val,mape,rmse,r2_adj,rsd,aic,bic
model_1_9_0,0.814344,0.578363,0.721707,0.094933,0.250302,0.051381,0.150734,0.308111,1.149556,0.318925,134.709178,225.787008
model_1_9_27,0.814344,0.578363,0.721707,0.094933,0.250302,0.051381,0.150734,0.308111,1.149556,0.318925,134.709178,225.787008
model_1_9_28,0.814344,0.578363,0.721707,0.094933,0.250302,0.051381,0.150734,0.308111,1.149556,0.318925,134.709178,225.787008
model_1_9_29,0.814344,0.578363,0.721707,0.094933,0.250302,0.051381,0.150734,0.308111,1.149556,0.318925,134.709178,225.787008
model_1_9_30,0.814344,0.578363,0.721707,0.094933,0.250302,0.051381,0.150734,0.308111,1.149556,0.318925,134.709178,225.787008
model_1_9_31,0.814344,0.578363,0.721707,0.094933,0.250302,0.051381,0.150734,0.308111,1.149556,0.318925,134.709178,225.787008
model_1_9_32,0.814344,0.578363,0.721707,0.094933,0.250302,0.051381,0.150734,0.308111,1.149556,0.318925,134.709178,225.787008
model_1_9_33,0.814344,0.578363,0.721707,0.094933,0.250302,0.051381,0.150734,0.308111,1.149556,0.318925,134.709178,225.787008
model_1_9_34,0.814344,0.578363,0.721707,0.094933,0.250302,0.051381,0.150734,0.308111,1.149556,0.318925,134.709178,225.787008
model_1_9_35,0.814344,0.578363,0.721707,0.094933,0.250302,0.051381,0.150734,0.308111,1.149556,0.318925,134.709178,225.787008


DataFrame salvo em ../results/metrics_1_9
